In [1]:
import boto3
import pandas as pd
import json
import matplotlib.pyplot as plt
import time
import re

# Configure pandas to display all columns and their full content without truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Do not truncate column contents
pd.set_option('display.expand_frame_repr', False)  # Avoid wrapping to the next line

## Load previously generated files

In [2]:
# Define your S3 bucket and file key
bucket_name = 'sagemaker-studio-423623869859-3no3d9ie4hx'
file_key_extracted_entities = 'extracted_entities_in_questions_train.json'
file_key_parsed_schemas = 'database_parsed_schemas.json'
file_key_training_data = 'train_spider.json'

# Initialize the S3 client
s3_client = boto3.client('s3')

# Load the file from S3
obj_extracted_entities = s3_client.get_object(Bucket=bucket_name, Key=file_key_extracted_entities)
questions_entities = json.load(obj_extracted_entities['Body'])

obj_parsed_schemas = s3_client.get_object(Bucket=bucket_name, Key=file_key_parsed_schemas)
all_schemas = json.load(obj_parsed_schemas['Body'])

obj_training_data = s3_client.get_object(Bucket=bucket_name, Key=file_key_training_data)
json_train = json.load(obj_training_data['Body'])

In [3]:
len(questions_entities)

6962

In [4]:
len(all_schemas)

148

In [5]:
len(json_train)

7000

In [6]:
question_entities_tables_list = []

for ele in json_train:
    question = ele['question']
    entities_for_tables = questions_entities[question]['entities_for_tables']
    entities_for_columns = questions_entities[question]['entities_for_columns']
    sql = ele['query']
    question_entities_tables_list.append([question, entities_for_tables, entities_for_columns, sql])

In [7]:
df_question_entities_tables = pd.DataFrame(question_entities_tables_list,\
                                           columns=['question', 'entities_for_tables', 'entities_for_columns', 'query'])

## extract table names from queries

In [8]:
import re
import pandas as pd

def extract_tables_from_sql(sql_query):
    """
    Extracts table names from a given SQL query string.

    Parameters:
    - sql_query (str): A string containing an SQL query.

    Returns:
    - list: A list of table names extracted from the query.
    """
    # Regular expression to capture table names after FROM, JOIN, and similar keywords
    table_regex = re.compile(r'(?i)\bFROM\b\s+([a-zA-Z_][a-zA-Z0-9_\-\.]*|\[[^\]]+\]|\`[^\`]+\`)|\bJOIN\b\s+([a-zA-Z_][a-zA-Z0-9_\-\.]*|\[[^\]]+\]|\`[^\`]+\`)')
    
    # Find all matches
    matches = table_regex.findall(sql_query)
    
    # Extract table names from matches and remove duplicates
    tables = []
    for match in matches:
        # Matches return tuples; extract the first non-empty group
        table = match[0] or match[1]
        if table:
            # Strip quotes or brackets around the table name
            table = table.strip('`[]')
            tables.append(table)
    
    return list(set(tables))  # Remove duplicates, if any


# Apply the function to the 'sql' column and create a new column 'tables'
df_question_entities_tables['tables'] = df_question_entities_tables['query'].apply(extract_tables_from_sql)

In [9]:
df_question_entities_tables['entities'] = df_question_entities_tables['entities_for_tables'] + df_question_entities_tables['entities_for_columns']

In [10]:
df_question_entities_tables.head(20)

,question,entities_for_tables,entities_for_columns,query,tables,entities
0,How many heads of the departments are older than 56 ?,[departments],"[heads, age]",SELECT count(*) FROM head WHERE age > 56,[head],"[departments, heads, age]"
1,"List the name, born state and age of the heads of departments ordered by age.",[heads of departments],"[name, born state, age]","SELECT name , born_state , age FROM head ORDER BY age",[head],"[heads of departments, name, born state, age]"
2,"List the creation year, name and budget of each department.",[department],"[creation year, name, budget]","SELECT creation , name , budget_in_billions FROM department",[department],"[department, creation year, name, budget]"
3,What are the maximum and minimum budget of the departments?,[departments],"[budget, maximum, minimum]","SELECT max(budget_in_billions) , min(budget_in_billions) FROM department",[department],"[departments, budget, maximum, minimum]"
4,What is the average number of employees of the departments whose rank is between 10 and 15?,[departments],"[employees, rank]",SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15,[department],"[departments, employees, rank]"
5,What are the names of the heads who are born outside the California state?,[heads],"[names, born, California state]",SELECT name FROM head WHERE born_state != 'California',[head],"[heads, names, born, California state]"
6,What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?,"[departments, secretary]","[creation years, managed by, born, state, Alabama]",SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id JOIN head AS T3 ON T2.head_id = T3.head_id WHERE T3.born_state = 'Alabama',"[department, management, head]","[departments, secretary, creation years, managed by, born, state, Alabama]"
7,What are the names of the states where at least 3 heads were born?,"[states, heads]","[names, born]",SELECT born_state FROM head GROUP BY born_state HAVING count(*) >= 3,[head],"[states, heads, names, born]"
8,In which year were most departments established?,[departments],"[year, established]",SELECT creation FROM department GROUP BY creation ORDER BY count(*) DESC LIMIT 1,[department],"[departments, year, established]"
9,Show the name and number of employees for the departments managed by heads whose temporary acting value is 'Yes'?,"[departments, employees]","[name, number of employees, heads, temporary acting]","SELECT T1.name , T1.num_employees FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id WHERE T2.temporary_acting = 'Yes'","[department, management]","[departments, employees, name, number of employees, heads, temporary acting]"


In [11]:
# Save the DataFrame to a local CSV file
file_name = 'df_question_entities_tables.csv'
df_question_entities_tables.to_csv(file_name, index=False, header=True)

# Upload the CSV file to the specified S3 bucket
s3_client.upload_file(file_name, bucket_name, file_name)

print(f"File {file_name} successfully uploaded to s3://{bucket_name}/{file_name}")

File df_question_entities_tables.csv successfully uploaded to s3://sagemaker-studio-423623869859-3no3d9ie4hx/df_question_entities_tables.csv


## Create a pandas df with database names and table names

In [12]:
schema_table_list = []

for schema in all_schemas:
    tables = all_schemas[schema]
    for table in tables:
        schema_table_list.append([schema, table])
        
df_schema_table = pd.DataFrame(schema_table_list, columns=['database', 'table'])

## Process the database names and table names by replacing _ with space and removing numbers

In [13]:
df_schema_table['processed_database'] = df_schema_table['database'].apply(lambda x : ' '.join(x.split('_')))
df_schema_table['processed_table'] = df_schema_table['table'].apply(lambda x: ' '.join(segment for segment in x.split('_') if not segment.isdigit())
)

## Concatenate database names and table names. It makes sense because database names serve as context for table names

In [14]:
df_schema_table['database_and_table'] = df_schema_table['processed_database'] + ' ' + df_schema_table['processed_table']

In [15]:
# Save the DataFrame to a local CSV file
file_name = 'df_schema_table.csv'
df_schema_table.to_csv(file_name, index=False, header=True)

# Upload the CSV file to the specified S3 bucket
s3_client.upload_file(file_name, bucket_name, file_name)

print(f"File {file_name} successfully uploaded to s3://{bucket_name}/{file_name}")

File df_schema_table.csv successfully uploaded to s3://sagemaker-studio-423623869859-3no3d9ie4hx/df_schema_table.csv


In [16]:
df_schema_table.tail(10)

,database,table,processed_database,processed_table,database_and_table
739,workshop_paper,Acceptance,workshop paper,Acceptance,workshop paper Acceptance
740,wrestler,wrestler,wrestler,wrestler,wrestler wrestler
741,wrestler,Elimination,wrestler,Elimination,wrestler Elimination
742,yelp,business,yelp,business,yelp business
743,yelp,category,yelp,category,yelp category
744,yelp,user,yelp,user,yelp user
745,yelp,checkin,yelp,checkin,yelp checkin
746,yelp,neighbourhood,yelp,neighbourhood,yelp neighbourhood
747,yelp,review,yelp,review,yelp review
748,yelp,tip,yelp,tip,yelp tip


In [17]:
df_schema_table.loc[df_schema_table['table']=='Customers']

,database,table,processed_database,processed_table,database_and_table
178,cre_Drama_Workshop_Groups,Customers,cre Drama Workshop Groups,Customers,cre Drama Workshop Groups Customers
214,customer_complaints,Customers,customer complaints,Customers,customer complaints Customers
219,customer_deliveries,Customers,customer deliveries,Customers,customer deliveries Customers
232,customers_and_addresses,Customers,customers and addresses,Customers,customers and addresses Customers
237,customers_and_invoices,Customers,customers and invoices,Customers,customers and invoices Customers
248,customers_and_products_contacts,Customers,customers and products contacts,Customers,customers and products contacts Customers
255,customers_campaigns_ecommerce,Customers,customers campaigns ecommerce,Customers,customers campaigns ecommerce Customers
262,customers_card_transactions,Customers,customers card transactions,Customers,customers card transactions Customers
275,department_store,Customers,department store,Customers,department store Customers
311,driving_school,Customers,driving school,Customers,driving school Customers
